# Introduction to ![spark](pics/spark.png) using ![scala](pics/python.png)

### Checking the version of spark

In [ ]:
spark.version

### Create a dataframe

In [ ]:
myRange = spark.range(1000).toDF("range")

In [ ]:
myRange

### Some transformations

- Spark will not act on transformations.
- All transformations in Spark are lazy => we wait until an action is called
- Spark will create a DAG (Directed Acyclic Graph) and act upon the source data
- Spark will optimize the pipeline
- Examples of [transformations](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations): map, filter, join, groupBy, sortByKey ... etc

In [ ]:
div3 = myRange.where("range % 3 = 0")

### Action

- Trigger the computation on the logic transformation
- Examples of [actions](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions): reduce, count, collect, take, saveAsTextFile ... etc

In [ ]:
div3.count()

We can check the results using the Spark UI: http://localhost:4040/

In [ ]:
div3.filter(div3["range"] < 25).show()

In [ ]:
div3.filter("range > 10 AND range < 25").count()

### Loading some data

In [ ]:
data = spark.read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("data/titanic.csv")

If we are calling this data frequently, it is better to cache it for faster access.

In [ ]:
data.cache()

In [ ]:
data.count()

In [ ]:
# data.take(3)
data.head(3)

In [ ]:
# data.show(3, truncate=False)
# data.select("Survived", "Sex", "Age").show(10)
data.show(3)

In [ ]:
data.columns

In [ ]:
data.printSchema()

In [ ]:
# summary statistics about the data
# data.describe('Sex').show()
data.describe().show()

### DataFrames overview

 - __Immutable__: once created cannot be changed. we applying transformation to the existing DF, a new one will be created
 - __Lazy__: unless there is an action performed on the DF, no transformaton will be computed
 - __Distributed__

### Manipulating DataFrames (or SparkSQL)

- __sort()__ :

    - When we are using `sort`, spark will not perform anything on the data, because it is just a transformation. However, it will create a plan for when an action is called. We can use `explain` to see the plan.
    - When reading the `explain`, on top we have the end result and at the bottom is the data we start with.
    - Only when we call an action on the data frame, the entire DAG is computed as shown in the `explain` pipeline

In [ ]:
data.sort("Survived").explain()

In [ ]:
from pyspark.sql.functions import desc

data.sort(desc("Survived")).show(2)

### Manipulating DataFrames (or SparkSQL)

- __createOrReplaceTempView()__
    - Spark SQL will create a temporary table from your DataFrame, which you can query with normal SQL
    - The temporary table can be manipulated with DataFrame code also
    - There is no performance difference between SQL and DF code

In [ ]:
data.createOrReplaceTempView("titanic_data")

In [ ]:
# this is a SparSQL query

spark.sql("""
    SELECT Sex, Survived, count(Survived) as count FROM titanic_data GROUP BY Sex, Survived ORDER BY Sex
""").show()

In [ ]:
# this is a Spark DataFrame query
data.groupBy("Sex", "Survived").count().sort("Sex").show()

### Manipulating DataFrames (or SparkSQL)

- __crosstab__(*col1, col2*)
    - pairwise frequency (contigency table)

In [ ]:
data.crosstab("Sex", "Survived").show()

### Manipulating DataFrames (or SparkSQL)

- __distinct()__
    - this will return a new DF containing the distinct rows in the original DF

In [ ]:
data.select('Embarked').distinct().show()

### Manipulating DataFrames (or SparkSQL)

- __dropna__(*how='any', thresh=None, subset=None*)
    - this will return a new DF omitting the rows containing null values
    

- __fillna__(*value, subset=None*)
    - it will replace null values

In [ ]:
data.count(), data.dropna(subset="Embarked").count()
#data.count(), data.dropna().count()

In [ ]:
data.fillna("X", subset="Embarked").select("Embarked").distinct().show()

### Manipulating DataFrames (or SparkSQL)

- __filter__(*condition*)
    - this will filter rows given a certain condition

In [ ]:
# data.filter(data.Sex == "male").count()
# data.filter(data["Sex"] == "female").count()
data.filter(data.Age < 25).count()

### Manipulating DataFrames (or SparkSQL)

- __groupBy__(*\*cols*)
    - groups the specified columns and runs aggregations on it
    
    
- __agg__(*\*expression*)
    - aggregating on a DF

In [ ]:
data.groupBy("Sex").agg({"Age": "average"}).show()

In [ ]:
data.agg({"Age": "max"}).show()

In [ ]:
data.groupBy("Sex").count().show()

### Manipulating DataFrames (or SparkSQL)

Something more complex:
 - transform the data frame into RDD (resilient distributed dataset)
 - apply a mapping function to each row in the data frame
 - transform it back to a data frame
 - rename the column to `gender`
 - order descending by `gender` column

In [ ]:
def getGender(string):
    if(string == 'male'): return 0
    elif(string == 'female'): return 1
    else: return -1

In [ ]:
rdd = data.select("Sex").rdd.map(lambda x: getGender(x.Sex))
df = spark.createDataFrame(rdd, "int").withColumnRenamed('value', 'Gender')
df.orderBy(df.Gender.desc()).show(5)

### Manipulating DataFrames (or SparkSQL)

Same thing as above, but using UDF (user defined functions)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [ ]:
udf_gender = udf(lambda x: getGender(x))

new_data = data.withColumn('Gender', udf_gender(data.Sex))

In [ ]:
new_data.show()